# Select specific attributes from models
When trying to select a specific value for all the instances of a model we can use the following code. Suppose you have the following notification model: 

In [ ]:
from django.db import models
from django.contrib.auth.models import User, Group
from django.utils import timezone
from django.core.exceptions import ValidationError
from django.contrib.contenttypes.models import ContentType
from django.contrib.contenttypes.fields import GenericForeignKey

# Create your models here.
class Notification(models.Model):
    """Model for notifications

    Args:
        models (Model): Models django class
    """
    NOTIFICATIONS_TYPE = [
        ("user", "User"), 
        ("group", "Group"),
    ]

    sender = models.ForeignKey(User, on_delete=models.SET_NULL, null=True, related_name="sent_notifications") 
    notification_type = models.CharField(max_length=15, choices=NOTIFICATIONS_TYPE)
    user_target = models.ForeignKey(User, on_delete=models.CASCADE, null=True, blank=True, related_name="notifications")
    group_target = models.ForeignKey(Group, on_delete=models.CASCADE, null=True, blank=True, related_name="group_notifications")    
    notification_date = models.DateTimeField(auto_now_add=True)
    is_read = models.BooleanField(default=False) 
    application = models.CharField(max_length=255)
    url = models.CharField(max_length=500, null=True, blank=True)
    message = models.CharField(max_length=255, null=True, blank=True)
    is_read_from_windows = models.BooleanField(default=False)
    deleted = models.BooleanField(default=False)
    
    # Generic relationship to any model in the techtool 
    content_type = models.ForeignKey(ContentType, on_delete=models.CASCADE, null=True, blank=True)
    object_id = models.PositiveIntegerField(null=True, blank=True)
    content_object = GenericForeignKey('content_type', 'object_id')
    
    def is_recent(self):
        """Checks if the notification is less than 2 days
        """
        return timezone.now() - self.notification_date <= timezone.timedelta(days=2)

    def __str__(self):
        target = self.user_target.first_name if self.user_target else self.group_target.name
        return f"Notification from {self.sender} to {target}" 
    
    def clean(self):
        if not self.user_target and not self.group_target:
            raise ValidationError("The notification must be either a user or group as target")
    
    class Meta:
        ordering = ['-notification_date']
        verbose_name = 'Notification'
        verbose_name_plural = 'Notifications'
        
        
    


## Select specific values for the object id

In [ ]:
def select_object_ids(request):
    current_notifications = Notification.objects.filter(
        user=request.user, application="gojo"
    ).values_list('object_id', flat=True)
    return current_notifications

## Select unique elements

In [ ]:
def select_unique_elements(request):
    current_notifications = Notification.objects.filter(
        user=request.user, application="gojo"
    ).values_list('object_id', flat=True)
    unique_ids = list(set(current_notifications))
    return unique_ids